In [81]:
from IPython.core.display import HTML
HTML("""
<style>
div {font-family:Open Sans,sans-serif;}
h1 {color:white;background-color: #446a7f;padding:10px}
h2{color:black; background-color:#b3b6a0; padding:10px}
h3{color:#78BE21; background-color:#446a7f; padding:10px}
h4{color:white; background-color:#78BE21;padding:5px}
.info{background-color:}
</style>
""")


<h1>Linked Art - Reconciling Collections Data - Geographical Location</h1>

The Linked Art reconciliation exemplar provides a step through the process of reconciling geographical place names that occur in the title of artworks by the artist, John Ruskin. From Wikipedia:

<blockquote>"John Ruskin (8 February 1819 – 20 January 1900) was an English writer, philosopher, art critic and polymath of the Victorian era. He wrote on subjects as varied as geology, architecture, myth, ornithology, literature, education, botany and political economy."</blockquote>

John Ruskin travelled extensively in Europe and was a prolific artist, creating drawings of paintings whose titles often included place names for the locations depicted. 

#### Artwork Title contains Place Name
The title of the artworks has been recorded in the title field in many of the collection data records , and this has been used as the basis for the reconciliation process shown here.

#### OpenRefine Tool to Reconcile Data
The place names are reconciled with the <a href="https://www.getty.edu/research/tools/vocabularies/tgn/">Getty Thesaurus of Geographic Names (TGN)</a>, using the <a href="https://openrefine.org/">Open Refine</a> tool.

#### The Getty Thesaurus of Geographic Names (TGN)
Reconciliation with the <a href="https://www.getty.edu/research/tools/vocabularies/tgn/">Getty Thesaurus of Geographic Names (TGN)</a> has allowed additional information to be associated with the artwork: 
- an authoritative global identifier for the geographical location depicted 
- geographical coordinates

#### Input Data Files

The input files are Linked Art files created with the <a href="https://github.com/tgra/Linked-Art/blob/main/01-06-Transform-John-Ruskin.ipynb">`01-06-Transform-John-Ruskin`</a> Jupyter notebook.


### Further Reading

- The Getty Thesaurus of Geographic Names® Online (TGN) http://www.getty.edu/research/tools/vocabularies/tgn
- John Ruskin Wikipedia entry https://en.wikipedia.org/wiki/John_Ruskin



In [82]:
try:
    import ipywidgets as widgets
except:
    !pip install ipywidgets
    import ipywidgets as widgets

from ipywidgets import Layout, FileUpload



try:
    import xmltodict
except:
    !pip install xmltodict
    import xmltodict

try:
    import json
except:
    !pip install json
    import json 
    
    
try:
    import requests
except:
    !pip install requests
    import requests


## Reconciliation Process

1. Create CSV file from JSON-LD Linked Art files, including relevant properties
2. Identify place name in the artwork title
3. Use OpenRefine to Reconcile Place Names
5. Define representation in Linked Art for geographical coordinates of place depicted in artwork
6. Incorporate geographical coordinates representation into existing JSON-LD files

## 1. Create CSV file from Linked Art JSON-LD

To reconcile the place names in the artwork titles a CSV file will be created extracting relevant fields from the JSON-LD Linked Art files.

he first step is to create a CSV file from Linked Art JSON-LD files. 

This is achieved by iterating over the Linked Art JSON-LD files, extracting the `id` and `_label` fields, and saving them to a CSV file.

An extract of the resulting CSV file is shown below.

In [83]:
try:
    import os
except:
    !pip install os
    import os

try:
    import json
except:
    !pip install json
    import json 
    
try:
    import IPython
except:
    !pip install IPython
    import IPython   
    
from IPython.display import display, HTML, Javascript

try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

import csv

# Linked Art JSON-LD file location
file_dir = "./data/ruskin/output/json/"

# output CSV file
csv_file = "./data/ruskin/ruskin-places.csv"

titles = []
file_list=os.listdir(file_dir)

# iterate Linked Art JSON-LD files
for file in file_list:
    # read file and append to 
    with open( file_dir + file) as json_file:  
        artwork = json.load(json_file)
        if "_label" not in artwork:
            continue
        titles.append(
            {
            "id":artwork["id"], 
                "place" : artwork["_label"], 
                "place_modified": " ", 
                "coords": " "})

# create CSV file
with open(csv_file, 'w') as f:  
    w = csv.DictWriter(f, ["id","place","place_modified","coords"])
    w.writeheader()
    w.writerows(titles)
     
# display CSV file
df = pd.read_csv(csv_file,low_memory=False)
display(df)

,id,place,place_modified,coords
0,https://collections.ashmolean.org/collection/1...,Engraving of Ruskin's Drawing of the Petal Vau...,,
1,https://collections.ashmolean.org/collection/1...,Enlarged Study of a Prawn's Rostrum,,
2,https://www.harvardartmuseums.org/collections/...,Study of a Venetian Capital,,
3,https://collections.ashmolean.org/collection/1...,"Autumnal Cloud filling the Valley of Geneva, t...",,
4,https://collections.ashmolean.org/collection/1...,Axmouth Landslip from Dolands Farm,,
...,...,...,...,...
274,https://collections.ashmolean.org/collection/1...,"The Head of a Kite, from Life",,
275,https://www.harvardartmuseums.org/collections/...,Part of a Sketch of the Northwest Porch of St....,,
276,http://www.rijksmuseum.nl/nl/collectie/nl-RP-T...,"Gezicht op S. Anastasia te Verona, over de Adige",,
277,https://collections.ashmolean.org/collection/2...,Architectural detail: stone bracket,,


## 2. Identify Place Names in Title

The next step is to extract possible place names from the artwork title field, to help with the reconciliation process. A list of possible place names is used to help identify place names in the field. These are added to the `place_modified` column of the CSV file. 

An extract of the resulting CSV file is shown below.

In [84]:
df = pd.read_csv(csv_file,low_memory=False)

places2 = ["Florence","Bologna","Lucca","Alps","Oxford","Rome", 
           "Venice","Fribourg","Neuchâtel","Sestri","Visp","Chamonix",
           "Abbeville","Schaffhausen","Verona","Vorarlberg","Baden","Schaffhausen","Faido","Normandy","Genève","Geneva",
           "Gloucester","Basel","Luzern","Padua","Habsburg","Rhine","Zug","Aix-la-Chapelle","Siena","Mont Blanc","Lago di Como",
           "Bellinzona","Lake of Lecco"
          
          ]
places = {"Venezia":["Venice","Venetian","St Mark","St. Mark"],
         }

for index,row in df.iterrows():
    # check if any value in places2 is present in rowp
    for place in places2:
        if place in row["place"]:
            df.at[index,"place_modified"] = place
    
    for place in places["Venezia"]:
        if place in row["place"]:
            df.at[index,"place_modified"] = "Venezia"

# remove records where place_modified is blank
df = df[df.place_modified != " "]
df.to_csv(csv_file, index=False) 

df = pd.read_csv(csv_file,low_memory=False)
display(df)

,id,place,place_modified,coords
0,https://www.harvardartmuseums.org/collections/...,Study of a Venetian Capital,Venezia,
1,https://collections.ashmolean.org/collection/1...,"Autumnal Cloud filling the Valley of Geneva, t...",Geneva,
2,https://www.harvardartmuseums.org/collections/...,"Tom Tower, Christ Church, Oxford",Oxford,
3,https://www.harvardartmuseums.org/collections/...,Study of a Venetian Capital,Venezia,
4,https://www.tate.org.uk/art/artworks/13033,View of Bologna,Bologna,
...,...,...,...,...
102,https://collections.ashmolean.org/collection/1...,Sketch of the Oak Spray in Mantegna's Fresco o...,Padua,
103,https://www.nga.gov/collection/72870,The Garden of San Miniato near Florence,Florence,
104,https://www.harvardartmuseums.org/collections/...,Part of a Sketch of the Northwest Porch of St....,Venezia,
105,http://www.rijksmuseum.nl/nl/collectie/nl-RP-T...,"Gezicht op S. Anastasia te Verona, over de Adige",Verona,


## 3. Use OpenRefine to Reconcile Place Names

The next step is to use OpenRefine to match the values in the place_modified field, with place names in the name authority, The Getty Thesaurus of Geographic Names® Online (TGN).

OpenRefine is a tool for working with messy data, and includes support for reconciliation with external data such as name authorities.


### Method
    
The method used to reconcile the place names is as follows:
- Install OpenRefine
- Open OpenRefine in browser
- Create a project
- Upload the places CSV file
- Review data
- Choose the TGN service to reconcile data with
- Select matched place name
- Choose to create a new column with the entity identifier

Further description of the process including screenshots is shown below.


### Further reading
- OpenRefine https://openrefine.org
- The Getty Thesaurus of Geographic Names® Online (TGN) http://www.getty.edu/research/tools/vocabularies/tgn
    

### Open CSV file in OpenRefine and review data

<img src="docs/media/img/openrefine.png"/>


### Reconcile the data in the `place_modified` column

- Right-click on `place_modified` column header
- Select `Start reconciling`

<img width="600px" src="docs/media/img/openrefine2.png"/>

### Choose Service to Reconcile Data With

- Choose a service to reconcile data with from reconciliation services known to Wikidata - https://reconciliation-api.github.io/testbench/
- Getty vocab services was chosen due to the Getty Thesaurus of Geographic Names® Online (TGN) that's included https://www.getty.edu/research/tools/vocabularies/tgn/

<img width="600px" src="docs/media/img/reconcileserv.png"/>

### Review Reconciliation Search Results

- review reconciliation search results and select relevant match if found
- Create new column to hold the tgn identifiers

<img width="600px" src="docs/media/img/tgncol.png"/>

### Manual Reconciliation

Some additional manual reconciliation was required using the TGN search form at http://www.getty.edu/research/tools/vocabularies/tgn
    
<img width="600px" src="docs/media/img/tgn.png"/>
    

### Result - CSV file with TGN Identifiers
The result of the reconciliation process is a file containing a new column with TGN name authority identifiers for place names identified in the artwork title. The file is created further to the following steps:
- export CSV file from OpenRefine
- save as [data/ruskin/ruskin-places-rec.csv](data/ruskin/ruskin-places-rec.csv) `
- remove lines that do not have entry in the TGN column and save file

An extract of the CSV file is shown below:

In [85]:
file = "data/ruskin/ruskin-places-rec.csv" 

df = pd.read_csv(file,low_memory=False)
df = df[df.place_modified != "Sestri"]
df.to_csv(file, index=False) 

df = pd.read_csv(file,low_memory=False)
display(df)

,id,place,place_modified,tgn,coords
0,https://www.harvardartmuseums.org/collections/...,Study of a Venetian Capital,Venice,tgn/7018159,NaN
1,https://www.harvardartmuseums.org/collections/...,"Tom Tower, Christ Church, Oxford",Oxfordshire,tgn/7011931,NaN
2,https://www.harvardartmuseums.org/collections/...,Study of a Venetian Capital,Venice,tgn/7018159,NaN
3,https://www.tate.org.uk/art/artworks/13033,View of Bologna,Bologna,tgn/7003127,NaN
4,https://www.harvardartmuseums.org/collections/...,Fragment of the Alps,Alps,tgn/7007746,NaN
...,...,...,...,...,...
83,https://www.harvardartmuseums.org/collections/...,"Study of an Archivolt in Saint Mark's, Venice",Venice,tgn/7018159,NaN
84,https://collections.ashmolean.org/collection/1...,"Afternoon in Spring, with south Wind, at Neuch...",Neuchâtel,tgn/7003751,NaN
85,https://collections.ashmolean.org/collection/1...,Sketch of the Oak Spray in Mantegna's Fresco o...,Padua,tgn/7003085,NaN
86,https://www.harvardartmuseums.org/collections/...,Part of a Sketch of the Northwest Porch of St....,Venice,tgn/7018159,NaN


## Add Geographical Coordinates to CSV file
The next step is to associate geographical coordinates with the Linked Art artwork representations, using the TGN identifiers to query a web service:

- Get geocoordinates for TGN identifiers
- request JSON file from http://vocab.getty.edu/tgn/ using TGN identifier
- add geocoordinates to CSV file

The resulting CSV file with geocoordinates added is shown below.

In [42]:
display(HTML("<H2>Geographical coordinates retrieved from TGN web service:</H2>"))   

file = "./data/ruskin/ruskin-places-rec.csv" 
filecoord = "./data/ruskin/ruskin-places-rec-coords.csv" 

df = pd.read_csv(file,low_memory=False)
df['coords'] = df['coords'].astype(str)

display(HTML("<h3>Retrieving geocoordinates from vocab.getty.edu TGN API. Please wait for task to complete.</h3>"))

latprop = "http://www.w3.org/2003/01/geo/wgs84_pos#lat"
lngprop = "http://www.w3.org/2003/01/geo/wgs84_pos#long"


geocoords = pd.DataFrame({}, columns=['tgn', 'latlng', ])


for gid in df['tgn'].unique():
    print(".", end='')
    query = "http://vocab.getty.edu/tgn/" + gid.split("tgn/",1)[1] +"-place.json"
    json_data = requests.get(query).json()
    
    for record in json_data:
        lat = json_data[record][latprop][0]["value"]
        lng = json_data[record][lngprop][0]["value"]
        latlng = str(lat) + "," + str(lng)
        
       
        #append row to the dataframe
        geocoords = geocoords.append({'tgn':gid, 'latlng':latlng}, ignore_index=True)
        

display(geocoords)


,tgn,latlng
0,tgn/7018159,"45.438611,12.326667"
1,tgn/7011931,"51.75,-1.25"
2,tgn/7003127,"44.466667,11.433333"
3,tgn/7007746,"46.416667,10"
4,tgn/7003165,"44.033333,10.45"
5,tgn/7003168,"43.216667,11.4"
6,tgn/7018001,"47.25,9.9167"
7,tgn/1117004,"45.916667,9.316667"
8,tgn/7011128,"51.833333,-2.25"
9,tgn/7010587,"50.106602,1.832691"


### Update Records with Geographical Coordinates 

The following code 
- merges the dataframe from the CSV file `df` with the dataframe `geocoords` containing the geocoordinates.
- removes the column `coords` from the `df` dataframe
- renames the `latlng` column to `coords` in the `df` dataframe
- writes the dataframe `df` to a CSV file

Finally it displays a summary view of the dataframe


In [31]:

# merge dataframe with coords with dataframe from csv
df = df.merge(geocoords, on='tgn') 

# drop column coords
df = df.drop('coords', 1) # drop column coords

# rename column latlng to coords
df.rename(columns={'latlng': 'coords'}, inplace=True) # rename column tgn to coords

# drop rows that have na value in coords column
df.dropna(subset=['coords'])  

# write to CSV file
df.to_csv(filecoord, index=False)

display(HTML("<H4>CSV file with Geographical Coordinates</H4>"))
# display dataframe
display(df)


,id,place,place_modified,tgn,coords
0,https://www.harvardartmuseums.org/collections/...,Study of a Venetian Capital,Venice,tgn/7018159,"45.438611,12.326667"
1,https://www.harvardartmuseums.org/collections/...,Study of a Venetian Capital,Venice,tgn/7018159,"45.438611,12.326667"
2,https://collections.ashmolean.org/collection/1...,Part of the Base of a Pilaster in Santa Maria ...,Venice,tgn/7018159,"45.438611,12.326667"
3,https://www.tate.org.uk/art/artworks/13032,The North-West Angle of the Facade of St Mark’...,Venice,tgn/7018159,"45.438611,12.326667"
4,https://collections.ashmolean.org/collection/1...,"The Palazzo Contarini-Fasan, Venice",Venice,tgn/7018159,"45.438611,12.326667"
...,...,...,...,...,...
83,https://collections.ashmolean.org/collection/1...,Mountain study: view of the Alps (Aiguilles ne...,Aiguilles,tgn/1031704,"44.781489,6.869433"
84,https://www.harvardartmuseums.org/collections/...,Scene on the Rhine,Rhine,tgn/7012611,"51.98404,4.08245"
85,https://www.harvardartmuseums.org/collections/...,"Study of Portal and Carved Pinnacles, Cathedra...",Normandie,tgn/8711852,"49.133333,.216667"
86,https://collections.ashmolean.org/collection/1...,Evening in Autumn under the Castle of Habsburg,Habsburg,tgn/8703166,"47.462441,8.185123"


## Define Representation in Linked Art

The next step is to define a representation in Linked Art for geographical coordinates of place depicted in artwork. The relevant parts of the Linked Art model are: 
 - Depiction
 - Geospatial approximation
 - Depiction of place with approximate location


### Linked Art Data Model - Depiction

Many sorts of artwork depict things that can be pointed out in the artwork. These could be identifiable entities, such as a known Person or Object with a name or identifier, or unidentifiable (perhaps fictional) instances of a class of entity, such as a depiction of a battle but not any particular battle. For example a portrait depicts the person sitting for it, or a sketch of a generic landscape depicts a place even if it's not a particular, known location. The depiction pattern describes what is in the artwork's image.

This is modeled using the represents property on the VisualItem, which refers to the entity that is being depicted.

`{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://linked.art/example/object/34",
  "type": "HumanMadeObject",
  "_label": "Self Portrait",
    "shows": [
    {
      "type": "VisualItem",
      "represents": [
        {
          "type": "Place",
          "_label": "Artist"
        }
      ]
    }
  ]}`


### Geospatial Approximation

All recorded locations are approximate to some degree. It may be desirable to capture this approximation separately from the actual place, especially when that approximation is very uncertain. Especially if the place is the exact location of several events, and perhaps an address or other information is known, but not the exact geospatial coordinates.

Secondly, as a place is defined by exactly one definition, but there might be multiple approximations such as a polygon as well as the central point, the real place that an activity occured at can be related to multiple approximate places to capture these different approximations.

`{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://linked.art/example/place/4",
  "type": "Place",
  "_label": "True Auction House Location",
  "approximated_by": [
    {
      "type": "Place",
      "_label": "Auction House Location Approximation",
      "defined_by": "POINT(-0.0032937526703165 51.515107154846)"
    }
  ]
}`


### Depiction of Place with Approximate Location

`{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://linked.art/example/object/34",
  "type": "HumanMadeObject",
  "_label": "geographical place name",
    "shows": [
    {
      "type": "VisualItem",
      "represents": [
        {
          "type": "Place",
          "_label": "Lucca",
          "approximated_by": [
                {
                  "type": "Place",
                  "_label": "Lucca - Location Approximation",
                  "defined_by": "POINT(-0.0032937526703165 51.515107154846)"
                }
              ]
        }
      ]
    }
  ]}`




### Further reading

- Depiction https://linked.art/model/object/aboutness/#depiction

- Geospatial approximation https://linked.art/model/place/#geospatial-approximation

<h3>Visualisation - Geographical Coordinates of Place Depicted in Artwork</h3>
<p>The following code will create a visualisation of the Linked Art JSON-LD representation of geographical coordinates of a place depicted in an artwork. You can explore the representation by clicking on nodes to expand/contract them.</p>
<p>The visualisation is an SVG representation that uses D3.js and is a modified version of code available at https://json-ld.org/playground/jsonld-vis.js and possibly https://github.com/science-periodicals/jsonld-vis</p>

<div id='vis' style='height:100%;width:6000px'></div>

In [71]:
from IPython.core.display import Javascript

code2 = "var file = './data/examples/geolocation.json';"\
        "var selector = '#vis';" \
        "visjsonld(file, selector); "  

with open('./src/js/visld.js', 'r') as _jscript:
    code = _jscript.read() + code2

Javascript(code)

<IPython.core.display.Javascript object>

##  Add Place Name and Coordinates into Linked Art JSON-LD Files

The final step is to add place names and geocoordinates to the original Linked Art files. 

The updated Linked Art files, including the geocoordinates, will later be used in a storymap visualisation of the artworks of John Ruskin, mapping the artworks to the locations that they depict, using the geocoordinates.

The `cromulent` Python library is used to create the JSON-LD representation.

In [117]:
try:
    import cromulent 
except:
    !pip install cromulent
    import cromulent
    
from cromulent.model import factory

from cromulent.model import factory, Actor, Production, BeginningOfExistence, EndOfExistence, TimeSpan, Place
from cromulent.model import InformationObject, Phase, VisualItem 
from cromulent.vocab import Painting, Drawing,Miniature,add_art_setter, PrimaryName, Name, CollectionSet, instances, Sculpture 
from cromulent.vocab import aat_culture_mapping, AccessionNumber, Height, Width, SupportPart, Gallery, MuseumPlace 
from cromulent.vocab import BottomPart, Description, RightsStatement, MuseumOrg, Purchase
from cromulent.vocab import Furniture, Mosaic, Photograph, Coin, Vessel, Graphic, Enamel, Embroidery, PhotographPrint
from cromulent.vocab import PhotographAlbum, PhotographBook, PhotographColor, PhotographBW, Negative, Map, Clothing, Furniture
from cromulent.vocab import Sample, Architecture, Armor, Book, DecArts, Implement, Jewelry, Manuscript, SiteInstallation, Text, Print
from cromulent.vocab import TimeBasedMedia, Page, Folio, Folder, Box, Envelope, Binder, Case, FlatfileCabinet
from cromulent.vocab import HumanMadeObject,Tapestry,LocalNumber
from cromulent.vocab import Type,Set
from cromulent.vocab import TimeSpan, Actor, Group, Acquisition, Place
from cromulent.vocab import Production, TimeSpan, Actor
from cromulent.vocab import LinguisticObject,DigitalObject, DigitalService
from cromulent import reader

try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd
    
try:
    import os
except:
    !pip install os
    import os
    
try:
    import json
except:
    !pip install json
    import json 
    

    

from lib import linkedart as la

artwork = {}
cnt=1

# directory containing Rusking artworks represented in Linked Art JSON-LD
ruskindir = "data/ruskin/output/json"
file_list=os.listdir(ruskindir)

# directory that will contain updated Ruskin artwork representations including geo coords
storyvisdir = "data/ruskin/storyvis/json"

# file containing reconciled data with coordinates
filecoord = "./data/ruskin/ruskin-places-rec-coords.csv" 
# open file containing reconciled data with geo coordinates
dataframeGeo = pd.read_csv(filecoord,low_memory=False)

# for each linked art json file
for file in file_list:
    # open file
    with open( ruskindir + "/" + file) as json_file:
        
        # get json object from file object with json.load() https://www.geeksforgeeks.org/json-load-in-python/
        artwork = json.load(json_file)
        
        # if id field is in the id field of data file containing geographical coordinates, add update the file
        if artwork["id"] in dataframeGeo["id"].tolist():
            
            display(HTML("<h4>" + artwork["_label"] + "</h4>"))
            # get rows in dataframeGeo where id == artwork id from JSON-LD file
            # Access a group of rows and columns by label(s) or a boolean array
            # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html
            rows = dataframeGeo.loc[dataframeGeo['id'] == artwork["id"]]
            
            print("Matching row in geographical coordinates file for artwork")
            display(rows)
            
            # get first row https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html
            row=rows.iloc[0]
            
         
            
            # get place name and coords from geocoords file 
            placeName = row["place"]
            coords = row["coords"]
            # replace comma in coords with space
            coords = coords.replace(",", " ")
           
            # increment counter
            cnt = cnt+1
             
            # use cromulent to create Linked Art representation of place depicted
             # https://github.com/thegetty/crom
            
            approx_place = Place()
            approx_place._label = placeName
            approx_place.defined_by = "POINT(" + coords + ")"
            
            place = Place()
            place._label = placeName
            place.approximated_by = approx_place
            
            visualItem = VisualItem()
            visualItem.represents = place
            
            # append new representation to artwork json object
            artwork["shows"] = factory.toJSON(visualItem)
            
            print("Geographical coordinates representation to be added:")
            print(json.dumps(factory.toJSON(visualItem), indent=2))
            
            # open output file 
            text_file = open(storyvisdir + "/" + str(cnt) + ".json", "wt")
            
            # write to file and close
            n = text_file.write(json.dumps(artwork,indent=2))
            text_file.close()
            print("File updated" )

HTML("<h4>Files updated</h4>")

,id,place,place_modified,tgn,coords
0,https://www.harvardartmuseums.org/collections/...,Study of a Venetian Capital,Venice,tgn/7018159,"45.438611,12.326667"


,id,place,place_modified,tgn,coords
20,https://www.harvardartmuseums.org/collections/...,"Tom Tower, Christ Church, Oxford",Oxfordshire,tgn/7011931,"51.75,-1.25"


,id,place,place_modified,tgn,coords
1,https://www.harvardartmuseums.org/collections/...,Study of a Venetian Capital,Venice,tgn/7018159,"45.438611,12.326667"


,id,place,place_modified,tgn,coords
32,https://www.tate.org.uk/art/artworks/13033,View of Bologna,Bologna,tgn/7003127,"44.466667,11.433333"


,id,place,place_modified,tgn,coords
33,https://www.harvardartmuseums.org/collections/...,Fragment of the Alps,Alps,tgn/7007746,"46.416667,10"


...

,id,place,place_modified,tgn,coords
17,https://www.harvardartmuseums.org/collections/...,"Study of an Archivolt in Saint Mark's, Venice",Venice,tgn/7018159,"45.438611,12.326667"


,id,place,place_modified,tgn,coords
78,https://collections.ashmolean.org/collection/1...,"Afternoon in Spring, with south Wind, at Neuch...",Neuchâtel,tgn/7003751,"46.990867,6.797675"


,id,place,place_modified,tgn,coords
87,https://collections.ashmolean.org/collection/1...,Sketch of the Oak Spray in Mantegna's Fresco o...,Padua,tgn/7003085,"45.416667,11.883333"


,id,place,place_modified,tgn,coords
18,https://www.harvardartmuseums.org/collections/...,Part of a Sketch of the Northwest Porch of St....,Venice,tgn/7018159,"45.438611,12.326667"


,id,place,place_modified,tgn,coords
19,https://collections.ashmolean.org/collection/1...,Study of the Marble Inlaying on the Front of t...,Venice,tgn/7018159,"45.438611,12.326667"


## Example Linked Art JSON-LD including Geographical Identifier and Coordinates 

The following is an example JSON-LD Linked Art representation, updated to include geographical coordinates.
    
### Image Title: Study of the Marble Inlaying on the Front of the Casa Loredan, Venice

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Study_of_the_Marble_Inlaying_on_the_Front_of_the_Casa_Loredan.jpg/508px-Study_of_the_Marble_Inlaying_on_the_Front_of_the_Casa_Loredan.jpg"/>



<a href="http://ruskin.ashmolean.org/collection/8979/object/14491">Ashmolean Museum artwork page</a>


### JSON-LD Representation

In [118]:
print(json.dumps(artwork,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://collections.ashmolean.org/collection/159660",
  "type": "HumanMadeObject",
  "_label": "Study of the Marble Inlaying on the Front of the Casa Loredan, Venice",
  "identified_by": [
    {
      "id": "http://lod.example.org/museum/Identifier/WA.RS.RUD.022",
      "type": "Identifier",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300312355",
          "type": "Type",
          "_label": "Accession Number"
        }
      ],
      "content": "WA.RS.RUD.022"
    },
    {
      "id": "http://lod.example.org/museum/Identifier/159660",
      "type": "Identifier",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404621",
          "type": "Type",
          "_label": "Owner-Assigned Number"
        }
      ],
      "content": "159660"
    },
    {
      "id": "https://collections.ashmolean.org/collection/159660/primary-name",
      "type": "Name",
     

<h3>Visualisation -  Artwork Description with Geographical Coordinates of Place Depicted</h3>

If you'd like to view a different file change the value of `file` filepath in the code below (examples: 1.json .. 89.json)

In [119]:
from IPython.core.display import Javascript

code2 = "var file = './data/ruskin/storyvis/json/4.json';"\
        "var selector = '#vis2';" \
        "visjsonld(file, selector); "  

with open('./src/js/visld.js', 'r') as _jscript:
    code = _jscript.read() + code2

Javascript(code)

<IPython.core.display.Javascript object>

<div id='vis2' style='height:100%;width:6000px'></div>